# Prep and Setup

### Install Detectron2

In [ ]:
!pip install -q pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

### Download code from repository

In [ ]:
! git clone https://github.com/seqryan/HumpbackWhaleSegmentation.git

### Download dataset

In [ ]:
% cd /content/HumpbackWhaleSegmentation

# Read instructions to generate kaggle.json file: https://www.kaggle.com/general/74235

! pip install -q --upgrade --force-reinstall --no-deps kaggle

from google.colab import files
print("Upload kaggle.json")
files.upload() # Upload the kaggle.json to cwd

# create ~/.kaggle folder and move the file kaggle.json to this folder
! mkdir -p ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# download the dataset
# Ensure that you agree to the terms of the competition beflore downloading the dataset to avoid 403 - Forbidden error. 
# Competition URL: https://www.kaggle.com/c/whale-categorization-playground
! kaggle competitions download -c whale-categorization-playground

# unzip and move the data to dataset directory
! mkdir dataset
! unzip -q whale-categorization-playground.zip -d dataset
! rm whale-categorization-playground.zip # cleanup to save disk space

## Download annotations

In [ ]:
% cd /content/HumpbackWhaleSegmentation
!wget https://github.com/seqryan/HumpbackWhaleSegmentation/releases/download/v0.1/detectron2-whale-segmentation-annotations.zip
!unzip -q detectron2-whale-segmentation-annotations.zip -d .

### Set all relevant paths

In [37]:
% cd /content/HumpbackWhaleSegmentation

ANNOTATIONS_FILE_NAME = 'Whale_Segmentation.json'
DATASET_DIR = 'dataset'
SAVE_WEIGHTS_DIR = 'model_weights'
LOAD_WEIGHTS_DIR = 'model_weights' # change to differnt path if you are using pretrained weights
OUTPUT_DIR = 'segmented_dataset'

/content/HumpbackWhaleSegmentation


# Train Segmentation Model

In [ ]:
% cd /content/HumpbackWhaleSegmentation
! python run.py train -s $SAVE_WEIGHTS_DIR -a $ANNOTATIONS_FILE_NAME -d $DATASET_DIR

# Load pretrained weights and preview generate segmented images

In [ ]:
% cd /content/HumpbackWhaleSegmentation

!wget https://github.com/seqryan/HumpbackWhaleSegmentation/releases/download/v0.1/detectron2-whale-segmentation-weights.zip
!unzip -q detectron2-whale-segmentation-weights.zip -d .

LOAD_WEIGHTS_DIR = 'pretrained_weights' # change to differnt path if you are using pretrained weights

In [ ]:
% cd /content/HumpbackWhaleSegmentation

! python run.py save -l $SAVE_WEIGHTS_DIR -a $ANNOTATIONS_FILE_NAME -d $DATASET_DIR -o $OUTPUT_DIR
! zip -q -r segmented_dataset.zip $OUTPUT_DIR

### Preview segmented results

In [ ]:
from matplotlib import pyplot as plt
import random
from matplotlib.pyplot import figure
figure(num=None,figsize=(10,10),dpi=80,facecolor='w',edgecolor='k')
images_names = os.listdir(os.path.join(OUTPUT_DIR, 'train'))
plt.axis('off')

samples = 5
index = 0
for d in random.sample(images_names, samples):
  index += 1
  ax = plt.subplot(samples, 2, index)
  ax.imshow(cv2.imread(os.path.join(DATASET_DIR, 'train', d)))
  ax.axis('off')

  index += 1
  ax = plt.subplot(samples, 2, index)
  ax.imshow(cv2.imread(os.path.join(OUTPUT_DIR, 'train', d)))
  ax.axis('off')

plt.show()